In [1]:
!pip install transformers

In [2]:
from transformers import BertForQuestionAnswering, BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
import torch
print(torch.__version__)

1.11.0+cpu


In [5]:
from flask import Flask, render_template, request

In [6]:
app = Flask(__name__)

In [7]:
@app.route("/")
def home():
    return render_template('index.html')

In [8]:
@app.route("/predict",methods = ["GET","POST"])
def predict():
    if request.method == 'POST':
        text = request.form['text']
        ques = request.form['question']
        encoded_text = tokenizer.encode_plus(text=ques,text_pair=text)
        inputs = encoded_text['input_ids']
        sentence_embeddings = encoded_text['token_type_ids']
        tokens = tokenizer.convert_ids_to_tokens(inputs)
        outputs = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embeddings]))
        start_index = torch.argmax(outputs.start_logits).item()
        end_index = torch.argmax(outputs.end_logits).item()
        answer = ' '.join(tokens[start_index:end_index+1])
        corrected_ans = ''
        
        for word in answer.split():
            if word[:2] == '##':
                corrected_ans += word[2:]
            else:
                corrected_ans += ' ' + word
                
        return render_template('index.html',prediction_text=f"{corrected_ans}")

In [ ]:
if __name__ == "__main__":
    app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [07/Dec/2022 17:35:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 17:35:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 17:36:24] "POST /predict HTTP/1.1" 200 -
